In [ ]:
"""
Created on Fri Sep 18 14:29 2021

This is an example script to apply multimelt

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd  
from tqdm.notebook import trange, tqdm
import time
import multimelt.melt_functions as meltf
import multimelt.plume_functions as pf
import multimelt.analysis_functions as ana
import multimelt.useful_functions as uf
from multimelt.constants import *
import seaborn as sns
import os, glob
import distributed

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
inputpath_data= #path to folder containing file_other and file_conc
inputpath_mask = #path to folder containing file_isf_orig
inputpath_profiles = #path to folder containing file_TS_orig
outputpath_simple = #path to folder containing gammas_file_simple

inputpath_plumes =  #path to folder containing the file with plume characteristics
inputpath_boxes = #path to folder containing the file with box characteristics

outputpath_melt = #path to write out output

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#file containing 'corrected_isfdraft'
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc') #file containing 'isfdraft_conc'
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc') #file containing 'ISF_mask','latitude', 'longitude','front_bot_depth_avg', 'front_bot_depth_max','isf_name'
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc') #file containing 'theta_ocean' and 'salinity_ocean'
file_TS = file_TS_orig.sel(Nisf=large_isf)

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics.nc')

PREPARE DATA

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

SIMPLE PARAMS

In [ ]:
gammas_file_simple = xr.open_dataset(outputpath_simple_all+'linreg_experiment_results.nc') # from tuning
#######
favier_gammas = xr.DataArray(np.array([2.030e-5,99.32e-5,132.9e-5]),dims='param') # from Favier et al. 2019
favier_gammas = favier_gammas.assign_coords({'param': ['linear_local', 'quadratic_local', 'quadratic_mixed_mean']})
#favier_gammas = xr.DataArray(np.array([1.060e-5,36.23e-5,36.3e-5]),dims='param')
#favier_gammas = favier_gammas.assign_coords({'param': ['linear_local', 'quadratic_local', 'quadratic_mixed_mean']})
#######
lipscomb_values_mpery = xr.DataArray(np.array([[1.11*10**4,1.44*10**4,2.06*10**6],
                                              [4.95*10**4,1.59*10**5,5.37*10**6]]), dims={'calib': ['MeanAnt','PIGL'], 'param': ['quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_mean_locslope']})
lipscomb_values_mpery = lipscomb_values_mpery.assign_coords({'calib': ['MeanAnt','PIGL'], 'param': ['quadratic_local', 'quadratic_mixed_mean', 'quadratic_mixed_locslope']})

lipscomb_values_mpers = lipscomb_values_mpery/yearinsec #from Lipscomb et al. 2021/Jourdain et al. 2020

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 

nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in simple_param_types:
        
    for gamma_choice in ['tuned','favier','lipscomb']:
        if gamma_choice == 'tuned':
            gamma = gammas_file_simple['slope'].sel(param=mparam)
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=True)

            ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
            ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                
        elif gamma_choice == 'favier':
            if mparam in favier_gammas.param.values:
                gamma = favier_gammas.sel(param=mparam)
                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=False)
                
                ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                
        elif gamma_choice == 'lipscomb':
            if mparam in lipscomb_values_mpers.param.values:
                gamma = lipscomb_values_mpers.sel(param=mparam, calib='MeanAnt')
                ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, U_param=False)  
                
                ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')

PLUMES

In [ ]:
plume_param_types = ['lazero19', 'lazero19_modif']
nisf_list = geometry_info_1D.Nisf
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for mparam in plume_param_types:
    for gamma_choice in ['tuned']: #'orig'
        
        if gamma_choice == 'tuned':
            if mparam == 'lazero19':
                gamma = 0.00013964 #tuning with 4 runs
                E0 = 0.06025211 #tuning with 4 runs
                 
                
            elif mparam == 'lazero19_modif':
                gamma = 0.00020468 #tuning with 4 runs
                E0 = 0.08860515 #tuning with 4 runs
                
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, verbose=True)
            
        elif gamma_choice == 'orig':
            
            gamma = gamma_eff_T_lazero
            E0 = E0_lazero
                
            ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma, E0=E0, verbose=True)
                
        ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
        ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')

BOXES

In [ ]:
tuned_box_gammas = xr.DataArray(np.array([0.82e-5,0.60e-5,0.82e-5]),dims='nD_config')
tuned_box_gammas = tuned_box_gammas.assign_coords({'nD_config': range(1,4)})
tuned_box_Cs = xr.DataArray(np.array([0.52e6,0.58e6,0.52e6]),dims='nD_config')
tuned_box_Cs = tuned_box_Cs.assign_coords({'nD_config': range(1,4)})

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 

for nD_config in range(1,4):
    
    for pism_version in ['yes','no']: 
        
        for picop_opt in ['yes','no']: 

            mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

            for gamma_choice in ['orig','tuned_ronja','tuned_clara']:

                if gamma_choice == 'orig':

                    C = C_pico
                    gamma = gT_star_pico
                    E0 = E0_lazero
                    
                    if picop_opt == 'yes':
                        gamma = gamma_eff_T_lazero

                    ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                          C=C, E0=E0, angle_option='appenB',
                                                          box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                          pism_version=pism_version, picop_opt=picop_opt)

                    ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                    ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                    
                elif gamma_choice == 'tuned_ronja':

                    C = 2.0e6
                    gamma = 5.5e-5
                    E0 = E0_lazero

                    if picop_opt == 'yes':
                        gamma = gamma_eff_T_lazero

                    ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                          C=C, E0=E0, angle_option='appenB',
                                                          box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                          pism_version=pism_version, picop_opt=picop_opt, T_corrections=True)
                    
                    ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                    ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')

                elif gamma_choice == 'tuned_clara':
                    
                    C = tuned_box_Cs.sel(nD_config=nD_config).values
                    gamma = tuned_box_gammas.sel(nD_config=nD_config).values
                    E0 = E0_lazero

                    if picop_opt == 'yes':
                        gamma = gamma_eff_T_lazero
                        
                    ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, T_S_profile, geometry_info_2D, geometry_info_1D, isf_stack_mask, mparam, gamma,
                                                          C=C, E0=E0, angle_option='appenB',
                                                          box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, box_tot_option='nD_config', 
                                                          pism_version=pism_version, picop_opt=picop_opt)
                    
                    ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')
                    ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'_'+gamma_choice+'_correctedTS.nc')